# Neural Rendering with Attention: An Incremental Improvement for Anime Character Animation
![image](https://transpchan.github.io/live3d/main.png)
This notebook is the official demo of [Neural Rendering with Attention: An Incremental Improvement for Anime Character Animation](https://github.com/transpchan/Live3D-v2). 
Pressing Ctrl+F9 will run all the code in sequence, and have fun!

本笔记本是Neural Rendering with Attention: An Incremental Improvement for Anime Character Animation 的官方demo。
按Ctrl+F9将依次运行全部的代码，玩得开心！

このノートブックは、Neural Rendering with Attention: An Incremental Improvement for Anime Character Animation の公式デモです。
Ctrl+F9 を押すと、すべてのコードが順番に実行され、楽しくなります!

이 노트북은 Neural Rendering with Attention: An Incremental Improvement for Anime Character Animation 의 공식 데모입니다.
Ctrl+F9를 누르면 모든 코드가 순서대로 실행되며 재미있습니다!

Dieses Notebook ist die offizielle Demo von Neural Rendering with Attention: An Incremental Improvement for Anime Character Animation.
Durch Drücken von Strg+F9 wird der gesamte Code nacheinander ausgeführt, und viel Spaß!


In [ ]:
#@title Download the code
#@markdown Download the code from https://github.com/transpchan/Live3D-v2 and install requirements.
!git clone https://github.com/transpchan/Live3D-v2.git
%cd Live3D-v2
!pip install -r requirements.txt

In [ ]:
#@title Download weights
#@markdown You may also replace the link to the latest weight on https://github.com/transpchan/Live3D-v2/releases.

!mkdir weights
#!wget https://github.com/transpchan/Live3D-v2/releases/download/checkpoints-2.1/checkpoints.zip
!wget https://github.com/transpchan/Live3D-v2/releases/download/checkpoints-2.2/checkpoints.zip
!unzip checkpoints.zip -d ./weights/

In [ ]:
#@title Choose a Sample. 
#@markdown Choose sample character sheets and UDP sequences. 



character = 'double_ponytail' #@param ['double_ponytail', 'short_hair', 'self_defined']


#@markdown **Caution**: The character sheets should be **PNG files with transparent background** and the character in them should be as similar as possible to the one given in the UDP sequence. Please you can try using https://github.com/KurisuMakise004/MMD2UDP (notebook:https://colab.research.google.com/github/KurisuMakise004/MMD2UDP/blob/main/COLAB.ipynb) to generate your own UDP sequence.


#@markdown **注意**：设定集必须是**透明背景的PNG文件**，里面的人物尽量和UDP序列中给出的人物相似。 请您尝试使用 MMD2UDP （ 笔记本：https://colab.research.google.com/github/KurisuMakise004/MMD2UDP/blob/main/COLAB.ipynb ）生成您自己的 UDP 序列。

#@markdown **注意**: キャラクター シートは、**背景が透明な PNG ファイル**である必要があり、その中の人物は、UDP シーケンスで指定されたものにできるだけ似ている必要があります。 MMD2UDP を使用して、独自の UDP シーケンスを生成してみてください。

#@markdown **주의**: 캐릭터 시트는 **배경이 투명한 PNG 파일**이어야 하며 인물은 UDP 시퀀스에서 주어진 것과 최대한 유사해야 합니다. MMD2UDP 를 사용하여 고유한 UDP 시퀀스를 생성할 수 있습니다.

#@markdown **Achtung**: Die Charakterbögen sollten **PNG-Dateien mit transparentem Hintergrund** sein und die Person darin sollte der in der UDP-Sequenz angegebenen Person so ähnlich wie möglich sein. Bitte versuchen Sie es mit MMD2UDP, um Ihre eigene UDP-Sequenz zu generieren.

In [ ]:
#@title Download the sample UDPs and Character Sheet. Skip if you want to upload your own

#@markdown Click start to download sample character sheets and UDP sequences.

#@markdown If you choose `self_defined` in the last step, please upload your own UDP sequences or pose image sequences to `/content/CoNR/poses/`,  and upload your own character sheets to `/content/CoNR/character_sheet/character/`.


!rm *.zip.*
!rm -r character_sheet/
!rm -r poses/
!mkdir character_sheet/
if character == 'short_hair':
  !wget https://github.com/transpchan/Live3D-v2/releases/download/samples/short_hair_images.zip
  !unzip -j  short_hair_images.zip -x '__MACOSX/*'  -d character_sheet/character/ 
elif character == 'double_ponytail':
  !wget https://github.com/transpchan/Live3D-v2/releases/download/samples/double_ponytail_images.zip
  !unzip -j  double_ponytail_images.zip -x '__MACOSX/*' -d character_sheet/character/
else:
  print("Please upload your character sheets to /content/CoNR/character_sheet/ ")
if character == 'short_hair':
  !wget https://github.com/transpchan/Live3D-v2/releases/download/samples/short_hair.zip
  !unzip -j  short_hair.zip -d poses/
elif character == 'double_ponytail':
  !wget https://github.com/transpchan/Live3D-v2/releases/download/samples/double_ponytail.zip
  !unzip -j double_ponytail.zip -d poses/ 
else:
  print("Please upload your UDP sequences or poses images to /content/CoNR/poses/ .")

In [ ]:
#@title Show all character sheets
from IPython.display import Image,display
from pathlib import Path
path ='./character_sheet/'
imgs = []
for file in Path(path).rglob('*.[PpWw][NnEe][GgBb]*'):
          imgs.append(Image(filename=str(file), width=200))
          
print("Num of character sheets:", len(imgs))
display(*imgs)

In [ ]:
#@title (Optional) Run UDP Detector
#@markdown This additional demo will show the results by running the udp detector on the character sheet. If you want to run it on the pose sequence, you need to change the code in train.py
!pip install open3d
!mkdir results
!python3 train.py --mode=test \
--dataloaders=2 \
--test_input_poses_images=./character_sheet/character/ \
--test_input_person_images=./character_sheet/ \
--test_output_dir=./results/ \
--test_checkpoint_dir=./weights/  \
--test_output_udp=True \
--test_output_video=False \
--test_pose_use_parser_udp=True



In [ ]:

#@title (Optional) Visualzie UDP detection results

if __name__ == "__main__":
    import numpy as np
    import open3d as o3d

    npz = np.load('./results/udp_0.npz', allow_pickle=True)
    print("img count:", npz["udp"].shape[0])
    a = np.moveaxis(npz["udp"][:, :, :, :], [2, 3], [0, 1]).reshape(-1, 4)
    img = np.moveaxis(npz["img"][:, :, :, :], [2, 3], [0, 1]).reshape(-1, 3)

    occulusion = (a[:, 3] > 0.90)

    xyz = a[occulusion, 0:3]
    rgb = img[occulusion, 0:3]
    print("points:", xyz.shape[0])
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz*[0.7, 0.25, 1])

    pcd.colors = o3d.utility.Vector3dVector(rgb)
    pcd.estimate_normals()
    pcd.orient_normals_consistent_tangent_plane(1)

    pcd2 = o3d.geometry.PointCloud()
    pcd2.points = o3d.utility.Vector3dVector([
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1],
    ])

    pcd2.paint_uniform_color([0.5, 0.5, 0.5])
    o3d.visualization.draw_plotly([pcd, pcd2])
    if False:
        print("Displaying pointcloud ...")
        o3d.visualization.draw([pcd])
    o3d.io.write_point_cloud("./pointcloud.ply", pcd)
    

img count: 4
points: 34836


In [ ]:
#@title Run Video Generation
#@markdown For sample data, this process may take about 40 minutes. You can stop earlier to get an shorter result (by clicking stop on the left).
!mkdir results
!python3 train.py --mode=test \
--dataloaders=2 \
--test_input_poses_images=./poses/ \
--test_input_person_images=./character_sheet/ \
--test_output_dir=./results/ \
--test_checkpoint_dir=./weights/ 


In [ ]:
#@title Convert video format for display
#@markdown `output.mp4` is the output with black background.  `output_adobe_premiere.mov` is the output with transparent background.
!ffmpeg -r 30 -y -i ./results/%d.png  -c:v qtrle output_adobe_premiere.mov 
!ffmpeg -r 30 -y -i ./results/%d.png  -c:v libx264 -strict -2 -pix_fmt yuv420p   output.mp4 

In [ ]:
#@title Play the generated video!
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 600):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
show_video('output.mp4')